In [ ]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

**Locate Data in your Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataLocation = "/content/drive/My Drive/COS 711/Assignments/Assignment 3/data/"

train=pd.read_csv(dataLocation + "Train.csv")
test=pd.read_csv(dataLocation + "Test.csv")
sample_sub=pd.read_csv(dataLocation + "sample_sub.csv")

# Inspect Data

In [ ]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [ ]:
test.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press
0,ID_test_0,C,"23.96666667,22.88333333,22.28333333,21.4833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.6898333329999999,0.722333333,0.7755833329999...","290.1352379,328.47011289999995,323.5730796,282...","0.6333333329999999,0.725,0.955,0.8725,0.6925,0...","87.64333333,87.67,87.70583333,87.75083333,87.7..."
1,ID_test_1,C,"18.99166667,19.70833333,20.95833333,22.7833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.99975,0.99725,0.956333333,0.851583333,0.7775...","295.2350543,307.6569135,313.7360236,311.518385...","0.640833333,1.244166667,1.284166667,1.2875,1.3...","87.8925,87.95666667,87.9925,87.9925,87.9808333..."
2,ID_test_10,D,"25.11666667,25.08333333,24.40833333,23.575,22....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.760166667,0.733583333,0.78475,0.825833333,0....","77.06161145,95.48124997,61.44498258,89.4688360...","2.0133333330000003,1.3025,0.783333333,0.530833...","90.34,90.2975,90.305,90.35583333,90.4425,90.49..."
3,ID_test_100,C,"18.71666667,20.60833333,22.28333333,23.9833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.604,0.0,0.0,...","0.951416667,0.92575,0.8603333329999999,0.77883...","310.7509058,316.186793,324.8167326,327.2174992...","0.745,1.169166667,1.316666667,1.385,1.40083333...","87.98833333,88.06333333,88.07833333,88.0366666..."
4,ID_test_1000,D,"17.63333333,18.05833333,19.625,21.28333333,23....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.9,0.898333333,0.8725,0.831666667,0.820833332...","334.6054069,329.6848561000001,322.7668036,312....","0.620833333,0.5,0.456666667,0.558333333,1.0158...","90.515,90.5575,90.62333333,90.6825,90.67916667..."


## Handle Missing Data
We replace the missing values with -100.

In [ ]:
def parseTimeSeriesData(timeSeries):
    timeSeries = timeSeries.replace("nan","-100").split(",")
    parsedTimeSeries = np.array([float(timeSeries) for timeSeries in timeSeries])
    return parsedTimeSeries

def extractTimeSeries(pattern, featureName):
    return np.array(pattern[featureName].apply(lambda timeSeries: parseTimeSeriesData(timeSeries)).tolist())

## Transpose the data.
(121, 6) -> (6, 121)

In [ ]:
featureNames = ['temp', 'precip', 'rel_humidity', 'wind_dir', 'wind_spd', 'atmos_press']

trainData = np.array([extractTimeSeries(train, featureName) for featureName in featureNames]).transpose(1,2,0)
testData = np.array([extractTimeSeries(test, featureName) for featureName in featureNames]).transpose(1,2,0)
trainTargets = train['target'].values

# Data Preprocessing

## Encode the location of the data

In [ ]:
def encodeLocation(pattern):
    encoding = {
        'A':0,
        'B':1,
        'C':2,
        'D':3,
        'E':4
    }
    return pattern['location'].apply(lambda encoding:pattern[encoding])

trainLocations = encodeLocation(train).values
testLocations = encodeLocation(test).values

## Normalise Data
Using Z score normalisation

In [ ]:
allFeatures = np.concatenate([trainData, testData], axis = 0)

for item in range(allFeatures.shape[2]):
    validIndex = np.where(allFeatures[:,:,c] != -100)
    mean = allFeatures[:,:,item][validIndex].mean()
    std = allFeatures[:,:,item][validIndex].std()
    
    allFeatures[:,:,item][validIndex] -= mean
    allFeatures[:,:,item][validIndex] /= std

trainData = allFeatures[:trainLocations.shape[0]]
testData = allFeatures[trainLocations.shape[0]:]

# Data

In [ ]:
class DataSet():
    def __init__(self, patterns, target, indices=None, split='a'):
        if indices is None:
            self.patterns = patterns
            self.target = target
        else:
            self.patterns = patterns[indices]
            self.target = target[indices]
        
        self.split = split

## Logic

In [ ]:
from tensorflow import keras
from sklearn.model_selection import StratifiedKFold
from numpy import mean, std, min

foldsSplits = 10
epochs = 15

In [ ]:
def calculatePerformance(history):
    validationLosses = history.history['val_loss']
    return min(validationLosses)


In [ ]:
def trainModel(model: keras.models.Sequential, trainData: DataSet, valData: DataSet, epochs):
    history = model.fit(
        x=trainData.patterns,
        y=trainData.target,
        validation_data=(valData.patterns, valData.target),
        epochs=epochs,
        verbose=2,
    )
    score = calculatePerformance(history)
    return model, score

In [ ]:
def testAlgorithm(modelCreation):
    global foldsSplits
    global epochs
    kFold = StratifiedKFold(n_splits=foldsSplits, random_state=2020, shuffle=True)

    scores = []
    fold = 0
    for trainIndex, valIndex in kFold.split(trainData, trainLocations):
        print('Training fold:', fold)
        fold += 1
        dataSets = {'train': DataSet(trainData, trainTargets, trainIndex, split = 'train'),
              'validation': DataSet(trainData, trainTargets, valIndex, split = 'validation'),
              'test': DataSet(testData, None, split = 'validation')}

        inputSize = len(trainData[0][0])
        model = modelCreation(inputSize)
        model, score = trainModel(model, dataSets['train'], dataSets['validation'], epochs)
        print("Fold Score: ", score)
        scores.append(score)
    return scores
    

# Convolutional NN

In [ ]:
def createCNNModel(inputSize):

    kernelSize = inputSize + 2

    model: keras.models.Sequential = keras.models.Sequential()
    model.add(keras.layers.Conv1D(inputSize, kernelSize, activation=keras.activations.relu))
    model.add(keras.layers.MaxPool1D())
    model.add(keras.layers.Conv1D(32, kernelSize, activation=keras.activations.relu))
    model.add(keras.layers.MaxPool1D())
    model.add(keras.layers.Conv1D(64, kernelSize, activation=keras.activations.relu))
    model.add(keras.layers.Dense(64, keras.activations.relu))
    model.add(keras.layers.Dense(1, keras.activations.relu))

    model.compile(optimizer='adam',
        loss='mse',
        metrics=['mse'])
    
    return model

In [ ]:
cnnScores = testAlgorithm(createCNNModel)

Training fold: 0
Epoch 1/15
438/438 - 4s - loss: 2181.9961 - mse: 2181.9961 - val_loss: 1469.6099 - val_mse: 1469.6099
Epoch 2/15
438/438 - 4s - loss: 1634.4282 - mse: 1634.4282 - val_loss: 1381.9690 - val_mse: 1381.9690
Epoch 3/15
438/438 - 4s - loss: 1603.2919 - mse: 1603.2919 - val_loss: 1388.5226 - val_mse: 1388.5226
Epoch 4/15
438/438 - 4s - loss: 1588.0895 - mse: 1588.0895 - val_loss: 1335.8751 - val_mse: 1335.8751
Epoch 5/15
438/438 - 4s - loss: 1591.1191 - mse: 1591.1191 - val_loss: 1344.2921 - val_mse: 1344.2921
Epoch 6/15
438/438 - 4s - loss: 1552.3832 - mse: 1552.3832 - val_loss: 1404.8209 - val_mse: 1404.8209
Epoch 7/15
438/438 - 4s - loss: 1540.3865 - mse: 1540.3865 - val_loss: 1324.7272 - val_mse: 1324.7272
Epoch 8/15
438/438 - 4s - loss: 1531.8767 - mse: 1531.8767 - val_loss: 1316.0332 - val_mse: 1316.0332
Epoch 9/15
438/438 - 4s - loss: 1518.2869 - mse: 1518.2869 - val_loss: 1324.1537 - val_mse: 1324.1537
Epoch 10/15
438/438 - 4s - loss: 1505.7505 - mse: 1505.7505 - val

In [ ]:
print(mean(cnnScores))

1471.9548950195312


# LSTM NN

In [ ]:
def createLSTM(inputSize):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(32, activation='relu', return_sequences=True, input_shape=(121,6)))
    model.add(keras.layers.Dense(64, activation='sigmoid'))
    model.add(keras.layers.Dense(1, activation='relu'))
    
    model.compile(loss='mse', # keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer='adam',
        metrics=['mse'])
    return model

lstmScores = testAlgorithm(createLSTM)

Training fold: 0
Epoch 1/15
438/438 - 20s - loss: 4081.3333 - mse: 4081.3333 - val_loss: 2807.4050 - val_mse: 2807.4050
Epoch 2/15
438/438 - 21s - loss: 2437.9546 - mse: 2437.9546 - val_loss: 1779.8870 - val_mse: 1779.8870
Epoch 3/15
438/438 - 21s - loss: 1894.7593 - mse: 1894.7593 - val_loss: 1621.4983 - val_mse: 1621.4983
Epoch 4/15
438/438 - 21s - loss: 1827.3748 - mse: 1827.3748 - val_loss: 1607.9473 - val_mse: 1607.9473
Epoch 5/15
438/438 - 21s - loss: 1816.4807 - mse: 1816.4807 - val_loss: 1599.4346 - val_mse: 1599.4346
Epoch 6/15
438/438 - 21s - loss: 1808.3716 - mse: 1808.3716 - val_loss: 1598.8765 - val_mse: 1598.8765
Epoch 7/15
438/438 - 21s - loss: 1805.0896 - mse: 1805.0896 - val_loss: 1589.9752 - val_mse: 1589.9752
Epoch 8/15
438/438 - 21s - loss: 1798.4478 - mse: 1798.4478 - val_loss: 1584.4728 - val_mse: 1584.4728
Epoch 9/15
438/438 - 21s - loss: 1793.1252 - mse: 1793.1252 - val_loss: 1579.9156 - val_mse: 1579.9156
Epoch 10/15
438/438 - 21s - loss: 1786.9968 - mse: 1786.

In [ ]:
print(mean(lstmScores))

1740.3574951171875


# Comparison

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
def benefitOfTheDoubt(scores):
      legitScores = []
      for score in scores:
          if str(score) != 'nan':
              legitScores.append(score)
      replacement = mean(legitScores)
      newScores = []
      for score in scores:
          if str(score) == 'nan':
              newScores.append(replacement)
          else:
              newScores.append(score)
      return newScores


In [ ]:
def calculateTStatistic(data1, data2, alpha = 0.1, description = ""):
    out = []   
    stat, p = wilcoxon(data1, data2)    
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    out.append("\n" + description + "\tStatistics=" + str(round(stat,8)) + "\tP value= " + str(round(p,8)))
    if p > alpha:
        out.append("\n" + description + '\tSame distribution (fail to reject H0) at ' + str(alpha))
    else:
        out.append("\n" + description + '\tDifferent distribution (reject H0) at ' + str(alpha))
    return out

print(calculateTStatistic(
    benefitOfTheDoubt(cnnScores), 
    benefitOfTheDoubt(lstmScores))
)

Statistics=0.000, p=0.005
['\n\tStatistics=0.0\tP value= 0.00506203', '\n\tDifferent distribution (reject H0) at 0.1']


In [ ]:
def printStatistics(data, description = ""):
    print("Mean of ", description, " is: ", str(mean(data)))
    print("Std of ", description, " is: ", str(std(data)))

printStatistics(benefitOfTheDoubt(cnnScores), "CNN")
printStatistics(benefitOfTheDoubt(lstmScores), "LSTM")

Mean of  CNN  is:  1471.9548950195312
Std of  CNN  is:  123.77456204718196
Mean of  LSTM  is:  1740.3574951171875
Std of  LSTM  is:  127.48737550993646
